In [1]:
print('jmd shree ganesha')

jmd shree ganesha


In [2]:
from langchain.prompts import PromptTemplate
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.vectorstores import chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import ctransformers
import os

In [3]:
dataset = r"C:\Users\Admin\Desktop\Aster\Chatbot - DRS\data"

In [4]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf(dataset)

In [6]:
def text_splitter(data_pdf):
    splitter = RecursiveCharacterTextSplitter(chunk_size=150, chunk_overlap=10)
    chunks = splitter.split_documents(data_pdf)
    
    return chunks

In [7]:
text_chunks = text_splitter(extracted_data)

In [8]:
text_chunks

[Document(page_content='Digital Rhombus Studios is a premier experience design house specializing in Virtual Reality (VR), Mixed \nReality (MR), and Augmented Reality (AR).', metadata={'source': 'C:\\Users\\Admin\\Desktop\\Aster\\Chatbot - DRS\\data\\test data.pdf', 'page': 0}),
 Document(page_content='With a team of industry professionals boasting over two decades of experience, Digital Rhombus crafts', metadata={'source': 'C:\\Users\\Admin\\Desktop\\Aster\\Chatbot - DRS\\data\\test data.pdf', 'page': 0}),
 Document(page_content='high-value, realistic VR experiences designed to deliver a "WOW" factor and significant business ROI.', metadata={'source': 'C:\\Users\\Admin\\Desktop\\Aster\\Chatbot - DRS\\data\\test data.pdf', 'page': 0}),
 Document(page_content='Our diverse portfolio includes projects for domestic and international clientele, many of which are under \nstrict NDAs.', metadata={'source': 'C:\\Users\\Admin\\Desktop\\Aster\\Chatbot - DRS\\data\\test data.pdf', 'page': 0}),
 D

In [9]:
print(len(text_chunks))

64


In [10]:
def text_to_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = text_to_embeddings()

c:\Users\Admin\Desktop\Aster\Chatbot - DRS\drs\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Admin\Desktop\Aster\Chatbot - DRS\drs\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
persist_directory = 'drs_db'

vectordb = chroma.Chroma.from_documents(documents=text_chunks, embedding=embeddings, persist_directory=persist_directory)

In [14]:
vectordb.persist()
vectordb = None

In [15]:
vectordb = chroma.Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [16]:
retriever = vectordb.as_retriever()

In [17]:
docsearch = retriever.get_relevant_documents('What is Digital Rhombus Solutions?')

c:\Users\Admin\Desktop\Aster\Chatbot - DRS\drs\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [18]:
len(docsearch)
print(docsearch)

[Document(page_content='Digital Rhombus continually pushes the boundaries of VR technology, ensuring our clients benefit from \nthe latest innovations.', metadata={'page': 1, 'source': 'C:\\Users\\Admin\\Desktop\\Aster\\Chatbot - DRS\\data\\test data.pdf'}), Document(page_content='Digital Rhombus continually pushes the boundaries of VR technology, ensuring our clients benefit from \nthe latest innovations.', metadata={'page': 1, 'source': 'C:\\Users\\Admin\\Desktop\\Aster\\Chatbot - DRS\\data\\test data.pdf'}), Document(page_content='Digital Rhombus continually pushes the boundaries of VR technology, ensuring our clients benefit from \nthe latest innovations.', metadata={'page': 1, 'source': 'C:\\Users\\Admin\\Desktop\\Aster\\Chatbot - DRS\\data\\test data.pdf'}), Document(page_content='Digital Rhombus Studios is a premier experience design house specializing in Virtual Reality (VR), Mixed \nReality (MR), and Augmented Reality (AR).', metadata={'page': 0, 'source': 'C:\\Users\\Admin\\D

In [19]:
retriever = vectordb.as_retriever(search_kwargs={'k':4})

In [20]:
retriever.search_type

'similarity'

In [21]:
retriever.search_kwargs

{'k': 4}

In [22]:
prompt_template="""
Use the following pieces of information to answer the user's question. You are a helpful assistant who answers questions related to Digital Rhombus Studios.
If the question is not related to Digital Rhombus Studios, respond with "I don't know, it's out of my knowledge base."
If you don't know the answer, just say to visit their contact us page. Do not ask for the phone number.

Context: {context}

Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [23]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [24]:
llm=ctransformers.CTransformers(model=r"C:\Users\Admin\Desktop\Aster\FAQ Chatbot\FAQ-Chatbot\model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':2048,
                          'temperature':0.5,'context_length':4096})

In [25]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=vectordb.as_retriever(search_kwargs={'k': 5}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [26]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

c:\Users\Admin\Desktop\Aster\Chatbot - DRS\drs\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response :  Digital Rhombus Studios is a premier experience design house specializing in Virtual Reality (VR), Mixed 
Reality (MR), and Augmented Reality (AR).
Response :  They create immersive environments for various industries, including education, healthcare, aviation, and entertainment.
Response :  Digital Rhombus Studios uses a variety of technologies to create immersive environments for various industries, including education, healthcare, aviation, and entertainment. Some of the technologies they use include virtual reality (VR), augmented reality (AR), and mixed reality (MR).
Response :  VR stands for Virtual Reality, which refers to a computer-generated simulation of an environment that can be experienced and interacted with in a seemingly real or physical way. It is created by using a combination of hardware and software to generate a three-dimensional environment that can be perceived and manipulated in a way similar to the real world. VR technology has many applications, in